# QLoRA Fine-Tuning (Mistral-7B) avec HuggingFace

Ce notebook présente une implémentation de QLoRA (Dettmers et al., 2023) pour adapter un modèle decoder-only 7B à des tâches d'instruction-following.

Contraintes QLoRA respectées :
- Chargement du modèle en 4-bit (NF4) via bitsandbytes
- Poids du modèle base gelés (seuls les adaptateurs LoRA sont entraînables)
- Adaptateurs LoRA attachés via PEFT
- Entraînement en fp16 avec gradient accumulation

## 1) Imports et environnement

Installation des dépendances et vérification GPU pour environnement Kaggle/Colab.

In [1]:
# Install (Kaggle)
!pip install -q --upgrade pip
!pip install -q -U transformers accelerate peft datasets
!pip install -q bitsandbytes>=0.43.0

# Set CUDA library path
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')

# Imports
import torch
from datasets import Dataset, interleave_datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    set_seed,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

set_seed(42)
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))
    print("capability:", torch.cuda.get_device_capability(0))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.2 MB/s eta 0:00:0000:0100:01


2025-12-20 13:20:03.623097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766236804.067756      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766236804.197460      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766236805.245213      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766236805.245262      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766236805.245265      55 computation_placer.cc:177] computation placer alr

torch: 2.8.0+cu126
cuda available: True
gpu: Tesla T4
capability: (7, 5)


## 2) Chargement du modèle et tokenizer en 4-bit

QLoRA charge le modèle base en 4-bit (NF4 avec double quantization) et entraîne des adaptateurs LoRA en précision plus élevée (compute dtype fp16).

In [2]:
model_name = "mistralai/Mistral-7B-v0.1"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
if tokenizer.pad_token is None:
    # Pour le padding avec un modèle causal, on utilise souvent EOS comme PAD en fine-tuning.
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# QLoRA: quantization 4-bit (NF4) via bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Base model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    dtype=torch.float16,
)

# Pour l'entraînement (checkpointing), on coupe le cache KV
model.config.use_cache = False

# Geler explicitement les poids du modèle base (exigence QLoRA)
for p in model.parameters():
    p.requires_grad = False

print("Loaded base model in 4-bit.")

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Loaded base model in 4-bit.


## 3) Configuration LoRA et attachement des adaptateurs

Préparation du modèle quantifié pour l'entraînement k-bit puis attachement des adaptateurs LoRA sur les modules d'attention et MLP.

In [3]:
# Préparation standard PEFT pour l'entraînement k-bit (LayerNorm casting, etc.)
model = prepare_model_for_kbit_training(model)

# Configuration LoRA (paramètres raisonnables QLoRA)
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

# Attache les adaptateurs LoRA (PEFT)
model = get_peft_model(model, lora_config)

# Active gradient checkpointing pour réduire la VRAM
model.gradient_checkpointing_enable()

# Affiche le nombre de paramètres entraînables
model.print_trainable_parameters()

trainable params: 167,772,160 || all params: 7,409,504,256 || trainable%: 2.2643


## 4) Préparation des datasets et mélange 70/30

Instruction fine-tuning avec format `### Instruction` / `### Response`.

Sources de données :
- Task A : Alpaca (données générales d'instruction-following)
- Task B : GSM8K (problèmes mathématiques)

Mélange via `interleave_datasets` avec probabilités 70% Task B / 30% Task A.

In [ ]:
from datasets import load_dataset

alpaca = load_dataset("tatsu-lab/alpaca", split="train")
gsm8k = load_dataset("gsm8k", "main", split="train")

def map_alpaca(ex):
    instruction = ex["instruction"].strip()
    output = ex["output"].strip()
    
    if ex["input"] and ex["input"].strip():
        instruction = f"{instruction}\n\n{ex['input'].strip()}"
    
    return {
        "instruction": instruction,
        "response": output
    }

def map_gsm8k(ex):
    return {
        "instruction": ex["question"].strip(),
        "response": ex["answer"].strip()
    }

alpaca = alpaca.map(map_alpaca, remove_columns=alpaca.column_names)
gsm8k = gsm8k.map(map_gsm8k, remove_columns=gsm8k.column_names)

def clean_sample(ex):
    """Filter out low-quality samples"""
    instruction = ex["instruction"]
    response = ex["response"]
    
    if not instruction or not response:
        return False
    
    if len(instruction) < 10 or len(response) < 10:
        return False
    
    if len(instruction) > 2000 or len(response) > 2000:
        return False
    
    return True

print(f"Before cleaning - Alpaca: {len(alpaca)}, GSM8K: {len(gsm8k)}")
alpaca = alpaca.filter(clean_sample)
gsm8k = gsm8k.filter(clean_sample)
print(f"After cleaning - Alpaca: {len(alpaca)}, GSM8K: {len(gsm8k)}")

# Données réduites mais suffisantes pour voir une amélioration
alpaca = alpaca.shuffle(seed=42).select(range(10000))
gsm8k = gsm8k.shuffle(seed=42).select(range(4000))
print(f"\nDataset final - Alpaca: {len(alpaca)}, GSM8K: {len(gsm8k)} (Total: {len(alpaca) + len(gsm8k)})")

print("\n=== Exemple Alpaca ===")
print("Instruction:", alpaca[0]["instruction"][:200], "...")
print("Response:", alpaca[0]["response"][:200], "...")

print("\n=== Exemple GSM8K ===")
print("Instruction:", gsm8k[0]["instruction"][:200])
print("Response:", gsm8k[0]["response"][:200], "...")

Before cleaning - Alpaca: 52002, GSM8K: 7473
After cleaning - Alpaca: 49545, GSM8K: 7473

📊 Dataset final - Alpaca: 10000, GSM8K: 4000 (Total: 14000)

=== Exemple Alpaca ===
Instruction: Suggest a new kitchen ingredient for whipped cream. ...
Response: Vanilla extract. ...

=== Exemple GSM8K ===
Instruction: Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh
Response: Mimi has 2 x 12 = <<2*12=24>>24 sea shells.
Kyle has 24 x 2 = <<24*2=48>>48 sea shells.
Leigh has 48 / 3 = <<48/3=16>>16 sea shells.
#### 16 ...


In [14]:
PROMPT_TEMPLATE = """### Instruction\n{instruction}\n\n### Response\n{response}"""

ds_a = alpaca
ds_b = gsm8k

def format_example(batch):
    return {
        "text": [
            PROMPT_TEMPLATE.format(instruction=i, response=r)
            for i, r in zip(batch["instruction"], batch["response"])
        ]
    }

ds_a = ds_a.map(format_example, batched=True, remove_columns=ds_a.column_names)
ds_b = ds_b.map(format_example, batched=True, remove_columns=ds_b.column_names)

# Split train/eval (exemple)
ds_a_split = ds_a.train_test_split(test_size=0.2, seed=42)
ds_b_split = ds_b.train_test_split(test_size=0.2, seed=42)

train_mixed = interleave_datasets(
    [ds_b_split["train"], ds_a_split["train"]],
    probabilities=[0.7, 0.3],
    seed=42,
    stopping_strategy="all_exhausted",
)
eval_mixed = interleave_datasets(
    [ds_b_split["test"], ds_a_split["test"]],
    probabilities=[0.7, 0.3],
    seed=42,
    stopping_strategy="all_exhausted",
)

print("Train size:", len(train_mixed))
print("Eval size:", len(eval_mixed))
print("Sample:\n", train_mixed[0]["text"])

# Tokenisation pour causal LM
# ⚡⚡⚡ MAX LENGTH RÉDUIT POUR ACCÉLÉRER ⚡⚡⚡
max_length = 512  # Réduit de 1024 à 512 (4x plus rapide)
def tokenize(batch):
    out = tokenizer(
        batch["text"],
        truncation=True,
        max_length=max_length,
        padding=False,  # Pas de padding ici, le data collator s'en chargera dynamiquement
    )
    out["labels"] = out["input_ids"].copy()
    return out

tokenized_train = train_mixed.map(tokenize, batched=True, remove_columns=train_mixed.column_names)
tokenized_eval = eval_mixed.map(tokenize, batched=True, remove_columns=eval_mixed.column_names)

# Utiliser DataCollatorForSeq2Seq avec padding dynamique au lieu de DataCollatorForLanguageModeling
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, 
    model=model,
    padding=True,  # Active le padding dynamique
    pad_to_multiple_of=8  # Optionnel : optimise les performances sur GPU
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Train size: 26498
Eval size: 6842
Sample:
 ### Instruction
Group the following data points into two clusters using the K-Means clustering algorithm.

65, 55, 45, 35, 25, 15

### Response
Cluster 1: 65, 55, 45
Cluster 2: 35, 25, 15


Map:   0%|          | 0/26498 [00:00<?, ? examples/s]

Map:   0%|          | 0/6842 [00:00<?, ? examples/s]

## 5) Configuration du Trainer

Hyperparamètres QLoRA pour VRAM limitée : batch size réduit avec gradient accumulation, optimiseur 8-bit paged AdamW, fp16.

In [15]:
output_dir = "./mistral7b-qlora-taskab"

training_args = TrainingArguments(
    output_dir=output_dir,
    
    # ⚡⚡⚡ CONFIGURATION ULTRA-RAPIDE ⚡⚡⚡
    per_device_train_batch_size=8,           # Augmenté au maximum
    gradient_accumulation_steps=2,           # Réduit au minimum
    
    num_train_epochs=1,
    
    learning_rate=2e-4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    
    fp16=True,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    max_grad_norm=0.3,
    gradient_checkpointing=True,
    
    logging_steps=100,                       # Encore moins de logging
    eval_strategy="epoch",                   # ⚡⚡⚡ Évalue seulement à la fin
    save_strategy="epoch",                   # ⚡⚡⚡ Sauvegarde seulement à la fin
    save_total_limit=1,                      # Une seule sauvegarde
    load_best_model_at_end=False,            # ⚡⚡⚡ Désactivé pour gagner du temps
    
    report_to="none",
    remove_unused_columns=False,
    dataloader_num_workers=0,                # ⚡ Désactivé (peut causer des lenteurs)
    dataloader_pin_memory=False,             # ⚡ Désactivé
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    processing_class=tokenizer,
    data_collator=data_collator,
)
print("Trainer ready.")

Trainer ready.


## 6) Boucle d'entraînement

Seuls les paramètres LoRA sont mis à jour. Le modèle base 4-bit reste gelé.

In [16]:
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

eval_metrics = trainer.evaluate()
trainer.log_metrics("eval", eval_metrics)
trainer.save_metrics("eval", eval_metrics)

print("Done.")

Epoch,Training Loss,Validation Loss
1,0.269200,1.160354


***** train metrics *****
  epoch                    =         1.0
  total_flos               = 322895201GF
  train_loss               =      0.4987
  train_runtime            =  8:05:59.68
  train_samples_per_second =       0.909
  train_steps_per_second   =       0.057


***** eval metrics *****
  epoch                   =        1.0
  eval_loss               =     1.1604
  eval_runtime            = 0:35:38.98
  eval_samples_per_second =      3.199
  eval_steps_per_second   =        0.4
Done.


## 7) Sauvegarde du modèle

PEFT sauvegarde les adaptateurs LoRA et leur configuration. Le tokenizer est également sauvegardé.

In [17]:
# Sauvegarde des adaptateurs LoRA (PEFT) + tokenizer
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Saved to:", output_dir)

# Pour recharger plus tard :
# - Charger le modèle base 4-bit (mêmes bnb_config)
# - Charger l'adapter PEFT depuis output_dir (PeftModel.from_pretrained)

Saved to: ./mistral7b-qlora-taskab


In [20]:
import shutil

shutil.make_archive(
    "/kaggle/working/mistral7b-qlora-taskab",
    "zip",
    "/kaggle/working/mistral7b-qlora-taskab"
)


'/kaggle/working/mistral7b-qlora-taskab.zip'